In [166]:
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [167]:
GESTURE_SIZE = 10

In [168]:
user = str(1)
raw_data = pd.read_csv('../raw-data/vol' + user + '.csv')

In [169]:
raw_data = raw_data.sort_values(['FINGER','Timestamp'])  # Sort values by finger first, then timestamp

In [170]:
raw_data = pd.get_dummies(raw_data,columns=['BTN_TOUCH'],drop_first=True) # drop first to prevent perfect predictor (overfitting)

# Feature Extraction

In [171]:
features = pd.DataFrame()  # final DF for features (sent to ML model)

In [172]:
def extract_features(gesture):  # takes gestures and extracts features
    gesture.insert(len(gesture.columns), "X_Speed", 0)
    gesture.insert(len(gesture.columns), "X_Acceleration", 0)
    gesture.insert(len(gesture.columns), "Y_Speed", 0)
    gesture.insert(len(gesture.columns), "Y_Acceleration", 0)
    gesture.insert(len(gesture.columns), "Speed", 0)
    gesture.insert(len(gesture.columns), "Acceleration", 0)
    gesture.insert(len(gesture.columns), "Jerk", 0)
    gesture.insert(len(gesture.columns), "Ang_V", 0)
    gesture.insert(len(gesture.columns), "Path_Tangent", 0)
    gesture['X_Speed'] = (gesture.X - gesture.X.shift(1)) / (gesture.Timestamp - gesture.Timestamp.shift(1))
    gesture['Y_Speed'] = (gesture.Y - gesture.Y.shift(1)) / (gesture.Timestamp - gesture.Timestamp.shift(1))
    gesture['Speed'] = np.sqrt((gesture.X_Speed ** 2) + (gesture.Y_Speed ** 2))
    gesture['X_Acceleration'] = (gesture.X_Speed - gesture.X_Speed.shift(1)) / (
                gesture.Timestamp - gesture.Timestamp.shift(1))
    gesture['Y_Acceleration'] = (gesture.Y_Speed - gesture.Y_Speed.shift(1)) / (
                gesture.Timestamp - gesture.Timestamp.shift(1))
    gesture['Acceleration'] = (gesture.Speed - gesture.Speed.shift(1)) / (
                gesture.Timestamp - gesture.Timestamp.shift(1))
    gesture['Jerk'] = (gesture.Acceleration - gesture.Acceleration.shift(1)) / (
                gesture.Timestamp - gesture.Timestamp.shift(1))
    gesture['Path_Tangent'] = np.arctan2((gesture.Y - gesture.Y.shift(1)), (gesture.X - gesture.X.shift(1)))
    gesture['Ang_V'] = (gesture.Path_Tangent - gesture.Path_Tangent.shift(1)) / (
                gesture.Timestamp - gesture.Timestamp.shift(1))

    return gesture


In [154]:
temp = pd.DataFrame()

In [173]:
for row in raw_data.iterrows():
    row = pd.Series(row)
#     print(row)
    pd.concat(temp, row)

C:\Users\skigu\AppData\Local\Temp\ipykernel_12272\2481018653.py:4: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  pd.concat(temp, row)


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"